In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [6]:
pip install transformers


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 96.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 107.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 21.7 MB/s eta 0:00:00


In [8]:
pip install wandb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 47.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 KB 12.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 KB 21.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 KB 7.6 MB/s eta 0:00:00
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8807 sha256=9b9c3d8344a243aae17c248a7787b6c1e35c0b79f4819bbe052eada739006d8b
  Stored in directory: /root/.cache/pip/wheels/b7/0a/67/ada2a22079218c75a88361c0782855cc72aebc4d18d0289d05
Successfully built pathtools


In [9]:
import torch
from torch.utils.data import DataLoader
from transformers import (
    AutoTokenizer,
    AutoModelForMaskedLM,
    DataCollatorForLanguageModeling,
    Trainer,
    TrainingArguments,
)

# Load the tokenizer and the saved model
tokenizer = AutoTokenizer.from_pretrained("microsoft/codebert-base")
model = AutoModelForMaskedLM.from_pretrained("/content/drive/MyDrive/codebertcustom-main (1)/model")

# Load your custom dataset
code_names_file = "/content/drive/MyDrive/codebertcustom-main (1)/arma3/data/ready/code_names.txt"
descriptions_file = "/content/drive/MyDrive/codebertcustom-main (1)/arma3/data/ready/descriptions.txt"


with open(code_names_file, "r") as f:
    code_names = f.readlines()

with open(descriptions_file, "r") as f:
    descriptions = f.readlines()

# Tokenize the data
input_texts = [code.strip() for code in code_names]
target_texts = [desc.strip() for desc in descriptions]

inputs = tokenizer(input_texts, return_tensors="pt", padding=True, truncation=True)
targets = tokenizer(target_texts, return_tensors="pt", padding=True, truncation=True)

# Create a dataset
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, inputs, targets):
        self.inputs = inputs
        self.targets = targets

    def __len__(self):
        return len(self.inputs["input_ids"])

    def __getitem__(self, idx):
        input_ids = self.inputs["input_ids"][idx]
        attention_mask = self.inputs["attention_mask"][idx]
        target_ids = self.targets["input_ids"][idx]
        target_attention_mask = self.targets["attention_mask"][idx]

        return {
            "input_ids": input_ids,
            "attention_mask": attention_mask,
            "labels": target_ids,
            "labels_attention_mask": target_attention_mask,
        }

dataset = CustomDataset(inputs, targets)

# Create a DataLoader
data_loader = DataLoader(dataset, batch_size=2, shuffle=True)

# Set training arguments
training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/codebertcustom-main (1)/model",
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=2,
    save_steps=5000,
    save_total_limit=2,
    logging_dir="./logs",
    logging_steps=500,
)

# Define a data collator
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)

# Create a Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
    data_collator=data_collator,
)

# Train the model
trainer.train()

try:
    trainer.save_model("/content/drive/MyDrive/codebertcustom-main (1)/model")
except Exception as e:
    print(f"Error saving model: {e}")

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
500,3.959300
1000,3.758700
1500,3.307800
2000,3.363500
2500,2.836000
3000,2.901300
3500,2.875200
4000,4.120300


now im going to retrain the the saved model on the same dataset as above 

In [10]:
# Load the saved model
retrained_model = AutoModelForMaskedLM.from_pretrained("/content/drive/MyDrive/codebertcustom-main (1)/model")

# Create a new Trainer instance with the loaded model
retrainer = Trainer(
    model=retrained_model,
    args=training_args,
    train_dataset=dataset,
    data_collator=data_collator,
)

# Train the model again using the same dataset
retrainer.train()

# Save the retrained model
try:
    retrainer.save_model("/content/drive/MyDrive/codebertcustom-main (1)/retrained_model")
except Exception as e:
    print(f"Error saving retrained model: {e}")


Step,Training Loss
500,1.358500
1000,1.453700
1500,1.317100
2000,1.291200
2500,1.051000
3000,1.321000
3500,1.546700
4000,3.448800


now that is trained and saved in Retrained model folder im going to up the learning rate.

Using a learning rate of 1e-5 (which is equal to 0.00001) is a more reasonable choice for fine-tuning a deep learning model. This value falls within the typical range of learning rates used for such tasks (between 1e-4 and 1e-6).

In [ ]:
# Load the saved model from the specified directory
model_to_retrain = AutoModelForMaskedLM.from_pretrained("/content/drive/MyDrive/codebertcustom-main (1)/retrained_model")

# Create a new set of training arguments with the adjusted learning rate
adjusted_training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/codebertcustom-main (1)/retrained_model_adjusted_lr",
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=2,
    save_steps=5000,
    save_total_limit=2,
    logging_dir="./logs",
    logging_steps=500,
    learning_rate=1e-5,  # Set the learning rate to 1e-5 (stay between (1e-4/1e-6))
)

# Create a new Trainer instance with the loaded model and the updated training arguments
adjusted_lr_trainer = Trainer(
    model=model_to_retrain,
    args=adjusted_training_args,
    train_dataset=dataset,
    data_collator=data_collator,
)

# Train the model again using the same dataset
adjusted_lr_trainer.train()

# Save the model trained with the adjusted learning rate
try:
    adjusted_lr_trainer.save_model("/content/drive/MyDrive/codebertcustom-main (1)/retrained_model_adjusted_lr")
except Exception as e:
    print(f"Error saving model with adjusted learning rate: {e}")


/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss
500,0.326300
1000,0.365500
1500,0.367700
2000,0.193900
2500,0.147900
